In [324]:
import Bio.PDB
import numpy
import pandas as pd

In [325]:
AT14 = ['4YAY','4ZUD']
AT16 = ['6DO1','6OS0','6OS1','6OS2']
arr = ['6TKO','6U1N','6UP7','6PWC']

In [326]:
# min_dist = 2.18
# max_dist = 3.51
min_dist = 0
max_dist = 2
atoms = ['N','O','S','SE']

In [327]:
def calc_atom_dist(atom1, atom2) :
    diff_vector  = atom1.coord - atom2.coord
    return numpy.sqrt(numpy.sum(diff_vector * diff_vector))

def calc_dist_matrix(chain1, chain2) :
    num = 0
    out = pd.DataFrame()
    for res1 in chain1 :
        for atom1 in res1:
            if atom1.element in atoms:
                for res2 in chain2 :
                    for atom2 in res2:
                        if atom2.element in atoms:
                            dist = calc_atom_dist(atom1, atom2)
                            if((dist<max_dist) & (dist>min_dist)):
                                part = pd.DataFrame([
                                    [
                                        chain1.id, num, res1.id[1], atom1.name, dist
                                    ],
                                    [
                                        chain2.id, num, res2.id[1], atom2.name, dist
                                    ]
                                    ],
                                    columns=[
                                        'chain','Cluster_ID', 'res', 'atom', 'dist'
                                    ]
                                )
                                out=pd.concat([out,part])
                                num += 1
    return out

In [328]:
def get_contacts(pdb_code):
    pdb_filename = f"/home/kestr/AT1/pdb/2_aligned/arr/{pdb_code}.pdb"
    structure = Bio.PDB.PDBParser().get_structure(pdb_code, pdb_filename)
    model = structure[0]
    out = calc_dist_matrix(model["A"], model["R"])
    out['gpcr']=pdb_code
    out['arr']=pdb_code
    return out

In [329]:
def get_contacts2(pdb_code1,pdb_code2):
    pdb_filename = f"/home/kestr/AT1/pdb/2_aligned/cutted_arr/{pdb_code1}_{pdb_code2}.pdb"
    structure = Bio.PDB.PDBParser().get_structure(pdb_code, pdb_filename)
    model = structure[0]
    out = calc_dist_matrix(model["A"], model["R"])
    out['gpcr']=pdb_code1
    out['arr']=pdb_code2
    return out

In [330]:
total = pd.DataFrame()
for pdb_code in arr:
    total = pd.concat([total,get_contacts(pdb_code)])

/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'CRYST1' at line 239
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 415
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'CRYST1' at line 28
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 52
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'CRYST1' at line 101
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 250
  warnings.warn(
/home/kes

In [331]:
for AT1 in AT14+AT16:
    for arr_i in arr:
        total = pd.concat([total,get_contacts2(AT1,arr_i)])

/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 457
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 216
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 279
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 104
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 283
  warnings.warn(
/home/kestr/AT1/.venv/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 168
  warnings.warn(
/home/kestr/AT1/

In [332]:
total[['Cluster_ID','res']]=total[['Cluster_ID','res']].astype('int')

In [333]:
total

,gpcr,arr,chain,Cluster_ID,res,atom,dist
0,4YAY,6TKO,A,0,68,SG,1.502253
1,4YAY,6TKO,R,0,237,OD2,1.502253
0,4YAY,6TKO,A,1,69,N,1.772130
1,4YAY,6TKO,R,1,235,N,1.772130
0,4YAY,6TKO,A,2,69,O,0.512528
...,...,...,...,...,...,...,...
1,6OS2,6U1N,R,0,126,NH1,1.852042
0,6OS2,6UP7,A,0,65,NH1,0.719886
1,6OS2,6UP7,R,0,137,NH1,0.719886
0,6OS2,6PWC,A,0,70,O,0.947536


In [334]:
total.to_csv('/home/kestr/AT1/data/hydrogen_contacts.csv', index=False)

In [335]:
# total.to_csv('/home/kestr/AT1/data/charge_repulsion.csv', index=False)